In [1]:
#imports
import math
import random
import pandas as pd
import numpy as np
import os

In [28]:
data = pd.read_excel("Raw_Data/1119.xlsx", engine="openpyxl" ,header=2,index_col=0)

In [29]:
data

,CITY 1,CITY 2,PASSENGERS \nTO CITY 2,PASSENGERS \nFROM CITY 2,FREIGHT \nTO CITY 2,FREIGHT \nFROM CITY 2,MAIL \nTO CITY 2,MAIL \nFROM CITY 2
S.No.,,,,,,,,
1,ADAMPUR,DELHI,2600,2525,0,1.534,0,0
2,AGARTALA,BENGALURU,3690,2854,1.412,2.682,0,1.05
3,AGARTALA,CHENNAI,2280,1327,0.001,0.285,0,0.5
4,AGARTALA,COIMBATORE,-,3,-,0.078,-,0
5,AGARTALA,DELHI,5480,5399,0.247,13.672,0.513,3.655
...,...,...,...,...,...,...,...,...
532,VIJAYAWADA,VISAKHAPATNAM,3054,2810,0.0429,1.8948,0,0
SUB TOTAL,NaN,NaN,6460649,6449486,30157.5,28077.9,1155.65,1160.1
TOTAL,NaN,NaN,12910135,NaN,58235.3,NaN,2315.74,NaN
